In [1]:
# Import packages 
import pandas as pd
import numpy as np
import re
import os
import sqlite3
from function_load_large_csv import load_large_csv 
import objects
from pandasql import sqldf
from sqlalchemy import create_engine

# Extract

In [2]:
# Specify the path to your large CSV file
filename = objects.file_name

# Define selected columns (optional)
usecols = ['form_type', 'filer_committee_id_number', 
           'transaction_id','entity_type', 'contributor_last_name',
           'contributor_first_name', 'contributor_street_1',
           'contributor_street_2', 'contributor_city', 'contributor_state',
           'contributor_zip_code','election_other_description','contribution_date', 'contribution_amount', 'contribution_aggregate',
           'contribution_purpose_descrip', 'contributor_employer',
           'contributor_occupation', 'memo_code', 'memo_text_description']

# Call the function to load the CSV into a DataFrame
df_ab_19 = load_large_csv(filename, usecols=usecols)


In [3]:
df_ab_19.columns

Index(['form_type', 'filer_committee_id_number', 'transaction_id',
       'entity_type', 'contributor_last_name', 'contributor_first_name',
       'contributor_street_1', 'contributor_street_2', 'contributor_city',
       'contributor_state', 'contributor_zip_code',
       'election_other_description', 'contribution_date',
       'contribution_amount', 'contribution_aggregate',
       'contribution_purpose_descrip', 'contributor_employer',
       'contributor_occupation', 'memo_code', 'memo_text_description'],
      dtype='object')

In [4]:
df_ab_19.dtypes

form_type                        object
filer_committee_id_number        object
transaction_id                   object
entity_type                      object
contributor_last_name            object
contributor_first_name           object
contributor_street_1             object
contributor_street_2            float64
contributor_city                 object
contributor_state                object
contributor_zip_code             object
election_other_description      float64
contribution_date                object
contribution_amount             float64
contribution_aggregate          float64
contribution_purpose_descrip     object
contributor_employer             object
contributor_occupation           object
memo_code                       float64
memo_text_description            object
dtype: object

In [5]:
df_ab_19.head(15)

,form_type,filer_committee_id_number,transaction_id,entity_type,contributor_last_name,contributor_first_name,contributor_street_1,contributor_street_2,contributor_city,contributor_state,contributor_zip_code,election_other_description,contribution_date,contribution_amount,contribution_aggregate,contribution_purpose_descrip,contributor_employer,contributor_occupation,memo_code,memo_text_description
0,SA11AI,C00401224,SA11AI_001,IND,Maloney,Gerald,1205 Hillcrest Drive,NaN,Euless,TX,76039,NaN,2019-01-02,15.0,0.0,NaN,NaN,NaN,NaN,NaN
1,SA11AI,C00401224,SA11AI_002,ORG,NaN,Susan Pope Hays,24 Stanwich Ln.,NaN,Greenwich,CT,06830,NaN,2019-01-03,25.0,0.0,NaN,NaN,NaN,NaN,NaN
2,SA11AI,C00401224,SA11AI_003,IND,Cassidy,William,921A Ridgefield Circle,NaN,Clinton,MA,01510,NaN,2019-01-03,25.0,0.0,NaN,NaN,NaN,NaN,NaN
3,SA11AI,C00401224,SA11AI_004,ORG,NaN,Lee Quintanar,2007 Rhoda St.,NaN,Simi Valley,CA,93065,NaN,2019-01-03,25.0,0.0,NaN,NaN,NaN,NaN,NaN
4,SA11AI,C00401224,SA11AI_005,ORG,NaN,Elizabeth A Juarez,2735 Midvale Ave.,NaN,Los Angeles,CA,90064,NaN,2019-01-08,195.0,0.0,NaN,NaN,NaN,NaN,NaN
5,SA11AI,C00401224,SA11AI_006,IND,Warner,Barbara,2675 Hillcrest Ave.,NaN,West Somerville,MA,02144,NaN,2019-01-08,10.0,0.0,NaN,NaN,NaN,NaN,NaN
6,SA11AI,C00401224,SA11AI_007,IND,Schwaninger,Vicki,3800 Humphrey St.,NaN,Saint Louis,MO,63116,NaN,2019-01-08,15.0,0.0,NaN,NaN,NaN,NaN,NaN
7,SA11AI,C00401224,SA11AI_008,IND,Bradley,Roxanne,8440 S Warhawk Rd,NaN,Conifer,CO,80433,NaN,2019-01-08,15.0,0.0,NaN,NaN,NaN,NaN,NaN
8,SA11AI,C00401224,SA11AI_009,IND,Lothrop,Caroll,19 Merriam Brook Road,NaN,Surry,NH,03431,NaN,2019-01-15,10.0,0.0,NaN,NaN,NaN,NaN,NaN
9,SA11AI,C00401224,SA11AI_010,ORG,NaN,Ethel Batiste,371 North Eldorado St.,NaN,San Mateo,CA,94401,NaN,2019-01-22,20.0,0.0,NaN,NaN,NaN,NaN,NaN


In [6]:
df_ab_19.isna().sum()

form_type                              0
filer_committee_id_number              0
transaction_id                         0
entity_type                            0
contributor_last_name                883
contributor_first_name               102
contributor_street_1                   0
contributor_street_2            14223693
contributor_city                      48
contributor_state                      0
contributor_zip_code                   0
election_other_description      14223693
contribution_date                      0
contribution_amount                    0
contribution_aggregate                 0
contribution_purpose_descrip          83
contributor_employer               27149
contributor_occupation              5644
memo_code                       14223693
memo_text_description                 83
dtype: int64

The existence of null values in memo_text_description is concerning, since this ties contributions processed through a conduit to the eventual recipient, as is the existance of null values in contributor_employer and contributor_occuppation. The presence of null values in contributor_last_name and contributor_first_name could likely be remedied by coalescing the two fields. 

In [7]:
df_ab_19.contribution_amount.sum()

339510724.81999993

# Transform

In [8]:
committee_data_cleaning_df=pd.DataFrame({'memo_text_description':df_ab_19.memo_text_description.unique()})

committee_data_cleaning_df.head()

,memo_text_description
0,NaN
1,Contribution to ActBlue
2,Earmarked for DCCC (C00000935)
3,Earmarked for KAMALA HARRIS FOR THE PEOPLE (C0...
4,Earmarked for BERNIE 2020 (C00696948)


In [9]:
committee_data_cleaning_df['recipient_cmte_id'] = committee_data_cleaning_df['memo_text_description'].str.extract(r'(C\d{8})')

In [16]:
committee_data_cleaning_df[committee_data_cleaning_df['recipient_cmte_id'].isnull() & committee_data_cleaning_df['recipient_cmte_id_clean'].isnull()].to_csv('/Users/sarahmathey/Desktop/missing_cmte_data.csv')

In [25]:
committee_data_cleaning_df['recipient_cmte_id_clean'] = np.where(
    committee_data_cleaning_df['memo_text_description'].str.contains('COMMITTEE FOR HISPANIC CAUSES BUILDING OUR LEADERSHIP DIVERSITY'), 'C00365536'
    , np.where(committee_data_cleaning_df['memo_text_description'].str.contains('SARA GIDEON FOR MAINE ()'), 'C00709899'
    , np.where(committee_data_cleaning_df['memo_text_description'].str.contains('MJ FOR TEXAS ()'), 'C00649350'
    , np.where(committee_data_cleaning_df['memo_text_description'].str.contains('SINEMA FOR ARIZONA ()'), 'C00508804'  
    , np.where(committee_data_cleaning_df['memo_text_description'].str.contains('GRISSOM FOR U.S. SENATE ()'), 'C00710780' 
    , np.where(committee_data_cleaning_df['memo_text_description'].str.contains('SCHOLTEN FOR CONGRESS ()'), 'C00711317'  
    , np.where(committee_data_cleaning_df['memo_text_description'].str.contains('KATE FOR CONGRESS ()'), 'C00711630'  
    , np.where(committee_data_cleaning_df['memo_text_description'].str.contains('RUBANDO FOR CONGRESS ()'), 'C00707943'  
    , np.where(committee_data_cleaning_df['memo_text_description'].str.contains('KRISTINE SCHANBACHER FOR CONGRESS ()'), 'C00710491'  
    , np.where(committee_data_cleaning_df['memo_text_description'].str.contains('NIKKI FOSTER FOR OHIO ()'), 'C00710459'  
    , np.where(committee_data_cleaning_df['memo_text_description'].str.contains('SISTER DISTRICT PROJECT, INC. ()'), 'C00636944'
    , np.where(committee_data_cleaning_df['memo_text_description'].str.contains('BRIANA URBINA FOR CONGRESS ()'), 'C00705087'  
    , np.where(committee_data_cleaning_df['memo_text_description'].str.contains('SACRAMENTO COUNTY DEMOCRATIC CENTRAL COMMITTEE: UNITED CAMPAIGN COMMITTEE-FEDERAL'), 'C00274423'  
    , np.where(committee_data_cleaning_df['memo_text_description'].str.contains('TEAM JJ USA ()'), 'C00704551'
    , np.where(committee_data_cleaning_df['memo_text_description'].str.contains('MIKE KATZ FOR PRESIDENT CAMPAIGN COMMITTEE ()'), 'C00703223'  
    , np.where(committee_data_cleaning_df['memo_text_description'].str.contains('MUES FOR MONTANA ()'), 'C00711978'  
    , np.where(committee_data_cleaning_df['memo_text_description'].str.contains('DETHRONE THE KING PAC ()'), 'C00695890'
    , np.where(committee_data_cleaning_df['memo_text_description'].str.contains('FEEL THE BERN DEMOCRATIC CLUB, ORANGE COUNTY ()'),'C00699215'  
    , np.where(committee_data_cleaning_df['memo_text_description'].str.contains('WIN THE FOURTH COLORADO ()'), 'C00658542'  
    , np.where(committee_data_cleaning_df['memo_text_description'].str.contains('END THE CORRUPTION ()'), 'C00693440'
    , np.where(committee_data_cleaning_df['memo_text_description'].str.contains('NY PROGRESSIVE ACTION PAC2 ()'), 'C00688762'  
    , np.where(committee_data_cleaning_df['memo_text_description'].str.contains('RESTORING OUR PRIORITIES EVERYWHERE ()'), 'C00699736'  
    , np.where(committee_data_cleaning_df['memo_text_description'].str.contains('INDIAN AMERICAN IMPACT FUND ()'), 'C00674127'
    , np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Contribution to ActBlue'),'C00401224'
    , np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for ACTBLUE TECHNICAL SERVICES (27-0160261)'), 'C00401224'
    , np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for NATIONAL COMMITTEE TO DRAFT OPRAH WINFREY FOR PRESIDENT OF THE UNITED STATES 2010'), 'C00666610'           
    , committee_data_cleaning_df['recipient_cmte_id']))))))))))))))))))))))))))


/var/folders/d1/22t7yn550bx1n_25p1j2h8b40000gn/T/ipykernel_38930/2079878001.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  , np.where(committee_data_cleaning_df['memo_text_description'].str.contains('SARA GIDEON FOR MAINE ()'), 'C00709899'
/var/folders/d1/22t7yn550bx1n_25p1j2h8b40000gn/T/ipykernel_38930/2079878001.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  , np.where(committee_data_cleaning_df['memo_text_description'].str.contains('MJ FOR TEXAS ()'), 'C00649350'
/var/folders/d1/22t7yn550bx1n_25p1j2h8b40000gn/T/ipykernel_38930/2079878001.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  , np.where(committee_data_cleaning_df['memo_text_description'].str.contains('SINEMA FOR ARIZONA ()'), 'C00508804'
/var/folde

In [21]:
committee_data_cleaning_df['recipient_fund_type'] = np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for -. Held pursuant to AOs 1977-16 and 1982-23'), 'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for AK-. Held pursuant to AOs 1977-16 and 1982-23'),	'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for AK-AL. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for AL-. Held pursuant to AOs 1977-16 and 1982-23'),	'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for AL-05. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for AR-. Held pursuant to AOs 1977-16 and 1982-23'),	'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for AR-01. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for AR-02. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for AR-03. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for AR-04. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for AZ-. Held pursuant to AOs 1977-16 and 1982-23'),	'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for AZ-05. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for AZ-06. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for AZ-08. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for CA-04. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for CA-22. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for CA-50. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for CO-. Held pursuant to AOs 1977-16 and 1982-23'),	'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for CO-04. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for FL-. Held pursuant to AOs 1977-16 and 1982-23'),	'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for FL-01. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for FL-15. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for GA-. Held pursuant to AOs 1977-16 and 1982-23'),	'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for GA-03. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for GA-07. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for GA-12. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for IA-. Held pursuant to AOs 1977-16 and 1982-23'),	'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for IA-04. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for ID-. Held pursuant to AOs 1977-16 and 1982-23'),	'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for IL-13. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for IN-05. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for IN-06. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for IN-08. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for KS-. Held pursuant to AOs 1977-16 and 1982-23'),	'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for KS-02. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for KY-. Held pursuant to AOs 1977-16 and 1982-23'),	'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for KY-06. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for LA-. Held pursuant to AOs 1977-16 and 1982-23'),	'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for ME-. Held pursuant to AOs 1977-16 and 1982-23'),	'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for MI-06. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for MI-07. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for MN-01. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for MN-08. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for MO-02. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for MS-. Held pursuant to AOs 1977-16 and 1982-23'),	'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for MT-. Held pursuant to AOs 1977-16 and 1982-23'),	'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for MT-AL. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for NC-. Held pursuant to AOs 1977-16 and 1982-23'),	'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for NC-02. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for NC-05. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for NC-13. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for ND-. Held pursuant to AOs 1977-16 and 1982-23'),	'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for ND-AL. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for NE-. Held pursuant to AOs 1977-16 and 1982-23'),	'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for NE-01. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for NE-02. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for NJ-04. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for NM-. Held pursuant to AOs 1977-16 and 1982-23'),	'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for NV-02. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for NY-01. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for NY-02. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for NY-23. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for NY-24. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for NY-27. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for OH-01. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for OH-04. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for OH-10. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for OH-12. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for OH-15. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for OK-. Held pursuant to AOs 1977-16 and 1982-23'),	'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for OK-02. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for PA-. Held pursuant to AOs 1977-16 and 1982-23'),	'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for PA-01. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for PA-10. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for PA-16. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for SC-. Held pursuant to AOs 1977-16 and 1982-23'),	'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for SD-. Held pursuant to AOs 1977-16 and 1982-23'),	'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for TN-. Held pursuant to AOs 1977-16 and 1982-23'),	'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for TN-06. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for TX-. Held pursuant to AOs 1977-16 and 1982-23'),	'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for TX-01. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for TX-02. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for TX-03. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for TX-06. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for TX-10. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for TX-19. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for TX-21. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for TX-22. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for TX-23. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for TX-24. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for TX-31. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for UT-02. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for VA-05. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for VA-06. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for VA-09. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for WA-03. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for WI-. Held pursuant to AOs 1977-16 and 1982-23'),	'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for WI-05. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for WV-. Held pursuant to AOs 1977-16 and 1982-23'),	'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for WV-01. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for WV-02. Held pursuant to AOs 1977-16 and 1982-23'),'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for WY-. Held pursuant to AOs 1977-16 and 1982-23'),	'Nominee Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earkmarked for DFA RUBEN GALLEGO FOR AZ SENATE 2020 DRAFT FUND. Held pursuant to AO 2006-30'),	'Draft Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earkmarked for DITCH FUND AMY MCGRATH FOR KY SENATE 2020 DRAFT FUND. Held pursuant to AO 2006-30'),'Draft Fund'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earkmarked for PCCC RUBEN GALLEGO FOR AZ SENATE 2020 DRAFT FUND. Held pursuant to AO 2006-30'),'Draft Fund'
, 'NaN'
)))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))

In [22]:
committee_data_cleaning_df['recipient_fund_office'] = np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for -. Held pursuant to AOs 1977-16 and 1982-23'),	    'President'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for AK-. Held pursuant to AOs 1977-16 and 1982-23'),  'Senate'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for AK-AL. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for AL-. Held pursuant to AOs 1977-16 and 1982-23'),	'Senate'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for AL-05. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for AR-. Held pursuant to AOs 1977-16 and 1982-23'),  'Senate'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for AR-01. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for AR-02. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for AR-03. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for AR-04. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for AZ-. Held pursuant to AOs 1977-16 and 1982-23'),	'Senate'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for AZ-05. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for AZ-06. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for AZ-08. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for CA-04. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for CA-22. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for CA-50. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for CO-. Held pursuant to AOs 1977-16 and 1982-23'),	'Senate'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for CO-04. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for FL-. Held pursuant to AOs 1977-16 and 1982-23'),	'Senate'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for FL-01. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for FL-15. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for GA-. Held pursuant to AOs 1977-16 and 1982-23'),	'Senate'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for GA-03. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for GA-07. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for GA-12. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for IA-. Held pursuant to AOs 1977-16 and 1982-23'),	'Senate'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for IA-04. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for ID-. Held pursuant to AOs 1977-16 and 1982-23'),	'Senate'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for IL-13. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for IN-05. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for IN-06. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for IN-08. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for KS-. Held pursuant to AOs 1977-16 and 1982-23'),	'Senate'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for KS-02. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for KY-. Held pursuant to AOs 1977-16 and 1982-23'),	'Senate'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for KY-06. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for LA-. Held pursuant to AOs 1977-16 and 1982-23'),	'Senate'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for ME-. Held pursuant to AOs 1977-16 and 1982-23'),	'Senate'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for MI-06. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for MI-07. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for MN-01. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for MN-08. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for MO-02. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for MS-. Held pursuant to AOs 1977-16 and 1982-23'),	'Senate'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for MT-. Held pursuant to AOs 1977-16 and 1982-23'),	'Senate'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for MT-AL. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for NC-. Held pursuant to AOs 1977-16 and 1982-23'),	'Senate'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for NC-02. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for NC-05. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for NC-13. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for ND-. Held pursuant to AOs 1977-16 and 1982-23'),	'Senate'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for ND-AL. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for NE-. Held pursuant to AOs 1977-16 and 1982-23'),	'Senate'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for NE-01. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for NE-02. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for NJ-04. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for NM-. Held pursuant to AOs 1977-16 and 1982-23'),	'Senate'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for NV-02. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for NY-01. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for NY-02. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for NY-23. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for NY-24. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for NY-27. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for OH-01. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for OH-04. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for OH-10. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for OH-12. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for OH-15. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for OK-. Held pursuant to AOs 1977-16 and 1982-23'),	'Senate'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for OK-02. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for PA-. Held pursuant to AOs 1977-16 and 1982-23'),	'Senate'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for PA-01. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for PA-10. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for PA-16. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for SC-. Held pursuant to AOs 1977-16 and 1982-23'),	'Senate'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for SD-. Held pursuant to AOs 1977-16 and 1982-23'),	'Senate'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for TN-. Held pursuant to AOs 1977-16 and 1982-23'),	'Senate'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for TN-06. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for TX-. Held pursuant to AOs 1977-16 and 1982-23'),	'Senate'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for TX-01. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for TX-02. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for TX-03. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for TX-06. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for TX-10. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for TX-19. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for TX-21. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for TX-22. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for TX-23. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for TX-24. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for TX-31. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for UT-02. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for VA-05. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for VA-06. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for VA-09. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for WA-03. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for WI-. Held pursuant to AOs 1977-16 and 1982-23'),	'Senate'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for WI-05. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for WV-. Held pursuant to AOs 1977-16 and 1982-23'),	'Senate'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for WV-01. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for WV-02. Held pursuant to AOs 1977-16 and 1982-23'),'House'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earmarked for Democratic Nominee for WY-. Held pursuant to AOs 1977-16 and 1982-23'),	'Senate'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earkmarked for DFA RUBEN GALLEGO FOR AZ SENATE 2020 DRAFT FUND. Held pursuant to AO 2006-30'),	'Senate'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earkmarked for DITCH FUND AMY MCGRATH FOR KY SENATE 2020 DRAFT FUND. Held pursuant to AO 2006-30'),'Senate'
, np.where(committee_data_cleaning_df['memo_text_description'].str.contains('Earkmarked for PCCC RUBEN GALLEGO FOR AZ SENATE 2020 DRAFT FUND. Held pursuant to AO 2006-30'),'Senate'
, 'NaN'
)))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))

In [26]:
committee_data_cleaning_df.isnull().sum()

memo_text_description        1
recipient_cmte_id          135
recipient_cmte_id_clean    109
recipient_fund_type          0
recipient_fund_office        0
dtype: int64

In [29]:
committee_data_cleaning_df[committee_data_cleaning_df['recipient_cmte_id'].isnull()].head(25)

,memo_text_description,recipient_cmte_id,recipient_cmte_id_clean,recipient_fund_type,recipient_fund_office
0,NaN,NaN,C00365536,Nominee Fund,President
1,Contribution to ActBlue,NaN,C00401224,NaN,NaN
9,Earmarked for MJ FOR TEXAS (),NaN,C00649350,NaN,NaN
42,Earmarked for CHC BOLD PAC/COMMITTEE FOR HISPA...,NaN,C00365536,NaN,NaN
72,Earmarked for SINEMA FOR ARIZONA (),NaN,C00508804,NaN,NaN
82,Earmarked for SARA GIDEON FOR MAINE (),NaN,C00709899,NaN,NaN
91,Earmarked for Democratic Nominee for KY-. Held...,NaN,NaN,Nominee Fund,Senate
105,Earmarked for Democratic Nominee for -. Held p...,NaN,NaN,Nominee Fund,President
155,Earkmarked for DITCH FUND AMY MCGRATH FOR KY S...,NaN,NaN,Draft Fund,Senate
228,Earmarked for Democratic Nominee for ME-. Held...,NaN,NaN,Nominee Fund,Senate


In [24]:
committee_data_cleaning_df[committee_data_cleaning_df['recipient_cmte_id'].isnull() & committee_data_cleaning_df['recipient_cmte_id_clean'].isnull()]

,memo_text_description,recipient_cmte_id,recipient_cmte_id_clean,recipient_fund_type,recipient_fund_office
1,Contribution to ActBlue,NaN,NaN,NaN,NaN
9,Earmarked for MJ FOR TEXAS (),NaN,NaN,NaN,NaN
42,Earmarked for CHC BOLD PAC/COMMITTEE FOR HISPA...,NaN,NaN,NaN,NaN
72,Earmarked for SINEMA FOR ARIZONA (),NaN,NaN,NaN,NaN
82,Earmarked for SARA GIDEON FOR MAINE (),NaN,NaN,NaN,NaN
...,...,...,...,...,...
1492,Earmarked for INDIAN AMERICAN IMPACT FUND (),NaN,NaN,NaN,NaN
1498,Earmarked for Democratic Nominee for IN-06. He...,NaN,NaN,Nominee Fund,House
1501,Earmarked for Democratic Nominee for AZ-05. He...,NaN,NaN,Nominee Fund,House
1502,Earmarked for Democratic Nominee for WI-05. He...,NaN,NaN,Nominee Fund,House


In [32]:
df_ab_19=df_ab_19.merge(committee_data_cleaning_df, right_on='memo_text_description',left_on='memo_text_description',how='left',suffixes=['_og','_clean'])

In [35]:
df_ab_19.isnull().sum()

form_type                              0
filer_committee_id_number              0
transaction_id                         0
entity_type                            0
contributor_last_name                883
contributor_first_name               102
contributor_street_1                   0
contributor_street_2            14223693
contributor_city                      48
contributor_state                      0
contributor_zip_code                   0
election_other_description      14223693
contribution_date                      0
contribution_amount                    0
contribution_aggregate                 0
contribution_purpose_descrip          83
contributor_employer               27149
contributor_occupation              5644
memo_code                       14223693
memo_text_description                 83
recipient_cmte_id                2995620
recipient_cmte_id_clean            90870
recipient_fund_type                    0
recipient_fund_office                  0
dtype: int64

# Load

In [ ]:
database_file = 'fec.db'

engine = create_engine(f'sqlite:///{database_file}')

table_name = 'actblue_2019_my'
df_ab_19.to_sql(table_name, engine, if_exists='replace', index=False)

In [ ]:
# Query the DataFrame using SQL syntax
query = "select sum(contribution_amount) from actblue_2019_my;”
result = sqldf(query, locals())

# Print the query results
print(result)